<a href="https://colab.research.google.com/github/sgcortes/25_BathymetrySotoBarca/blob/main/v3_MLP_dataFromLocalFiles_03_2025_BATIMETRIA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimación de la Batimetría a partir de imagen UAV multiespectral. MLP (Multilayer Perceptron)
+Cota lámina de agua 212m

---
En la hoja excel adjunta (NorteEsteProf.xlsx) y en la en el hoja OesteProf.xlsx, se tienen datos de cotas del fondo ("z") de un embalse para puntos de posiciones (campos "x" e "y"). Además hay un conjunto de valores de niveles de reflectancia (campos: "red", "green", "blue", "red_edge", "nir") y de otros índices multiespectrales ("ratiobg", "ndwi", "kwlightate","ndvibarca") que han sido calculados a partir de combinaciones de las variables de reflectancia.

El objetivo es entrenar un modelo GradientBoost (sklearn) que relacione estas variables (excepto, "fid", "x", "y", "z") con la "Profundidad" (calculada como : 212-"z"). Los datos suministrados (alrededor de 16000 mas 3000) se dividirán en 60% para entrenamiento y 20% para test y 20% para validación.
al final del entrenamiento dame las métricas del resultado obtenido para el conjunto test y el del entrenamiento y valiadción por separado. Además crea tres gráficas de valores predichos frente a valores reales para la profundad para el conjunto test y para el de entrenamiento y validación. Dibuja también una gráfica de importancia de ls variables en barras horizontales y ordeandas de mayor a menor.

## Lectura de datos

In [3]:
# Instalar Optuna si no está instalado (ya que el entorno fue reiniciado)
!pip install optuna
!pip install openpyxl

In [ ]:
import time

# Iniciar temporizador
start_time = time.time()

In [4]:
import pandas as pd
#from google.colab import drive
# Montar Google Drive
#drive.mount('/content/drive')

# Definir la ruta del directorio en Google Drive
#path = "/content/drive/My Drive/Colab Notebooks/03_2025_BATIMETRIA-SOTO-BARCA/"

# Leer los archivos de Excel
file_norte_este = "NorteEsteProf.xlsx"
file_oeste = "OesteProf.xlsx"

df_norte_este = pd.read_excel(file_norte_este)
df_oeste = pd.read_excel(file_oeste)

# Mostrar las primeras filas de los DataFrames para verificar la carga
df_norte_este.head(), df_oeste.head()


FileNotFoundError: [Errno 2] No such file or directory: 'NorteEsteProf.xlsx'

In [ ]:
# Recargar librerías necesarias después del reinicio del entorno
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Definir las características y la variable objetivo
features = ["red", "green", "blue", "red_edge", "nir", "ratiobg", "ndwi", "kwlightate", "ndvibarca"]
target = "Profundidad"

# Unir ambos conjuntos de datos en un solo DataFrame
df_total = pd.concat([df_norte_este, df_oeste], ignore_index=True)

# Imputar valores nulos con la mediana de cada columna
df_total.fillna(df_total.median(), inplace=True)

# Separar en variables independientes (X) y la variable objetivo (y)
X = df_total[features]
y = df_total[target]

# Dividir en conjunto de entrenamiento (60%), test (20%) y validación (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizar las características para mejorar la estabilidad del entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# Definir la arquitectura de la red neuronal
mlp_model = MLPRegressor(hidden_layer_sizes=(128, 64, 32),
                         activation='relu',
                         solver='adam',
                         max_iter=1000,
                         random_state=42)

# Entrenar la red neuronal
mlp_model.fit(X_train_scaled, y_train)

# Realizar predicciones con la red entrenada
y_train_pred_mlp = mlp_model.predict(X_train_scaled)
y_test_pred_mlp = mlp_model.predict(X_test_scaled)
y_val_pred_mlp = mlp_model.predict(X_val_scaled)

# Calcular métricas para el modelo MLP
metrics_train_mlp = {
    "MAE": mean_absolute_error(y_train, y_train_pred_mlp),
    "MSE": mean_squared_error(y_train, y_train_pred_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_mlp)),
    "R²": r2_score(y_train, y_train_pred_mlp)
}

metrics_test_mlp = {
    "MAE": mean_absolute_error(y_test, y_test_pred_mlp),
    "MSE": mean_squared_error(y_test, y_test_pred_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_mlp)),
    "R²": r2_score(y_test, y_test_pred_mlp)
}

metrics_val_mlp = {
    "MAE": mean_absolute_error(y_val, y_val_pred_mlp),
    "MSE": mean_squared_error(y_val, y_val_pred_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_val, y_val_pred_mlp)),
    "R²": r2_score(y_val, y_val_pred_mlp)
}


In [ ]:
# Mostrar métricas
print("\nTrain:\n",metrics_train_mlp, "\nTest:\n",metrics_test_mlp,"\nValidation:\n", metrics_val_mlp)


### Análisis:
El modelo tiene un R² ≈ 0.74 en el conjunto de test, lo que indica que la red neuronal logra explicar una buena parte de la variabilidad de la profundidad.
El rendimiento en test y validación es similar al de entrenamiento, lo que indica que el modelo no está sobreajustado.
Sin embargo, en comparación con el Gradient Boosting, el desempeño parece un poco inferior.

### Gráficos predichos vs Reales

In [ ]:
import matplotlib.pyplot as plt

# Crear gráficos de valores predichos vs. valores reales
plt.figure(figsize=(18, 5))

# Gráfico para el conjunto de entrenamiento
plt.subplot(1, 3, 1)
plt.scatter(y_train, y_train_pred_mlp, alpha=0.5)
plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], color="red", linestyle="dashed")
plt.xlabel("Profundidad Real")
plt.ylabel("Profundidad Predicha")
plt.title("Entrenamiento: Valores Reales vs. Predichos")

# Gráfico para el conjunto de test
plt.subplot(1, 3, 2)
plt.scatter(y_test, y_test_pred_mlp, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="dashed")
plt.xlabel("Profundidad Real")
plt.ylabel("Profundidad Predicha")
plt.title("Test: Valores Reales vs. Predichos")

# Gráfico para el conjunto de validación
plt.subplot(1, 3, 3)
plt.scatter(y_val, y_val_pred_mlp, alpha=0.5)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color="red", linestyle="dashed")
plt.xlabel("Profundidad Real")
plt.ylabel("Profundidad Predicha")
plt.title("Validación: Valores Reales vs. Predichos")

plt.tight_layout()
plt.show()


Análisis de los gráficos:

En entrenamiento, los valores predichos siguen bastante bien la línea de identidad roja, lo que indica un buen ajuste.
En test y validación, se mantiene una tendencia similar, aunque con mayor dispersión, lo que sugiere que la red neuronal generaliza razonablemente bien.


### Importancia de las variables

In [ ]:
from sklearn.inspection import permutation_importance

# Calcular la importancia de las variables usando Permutation Feature Importance
perm_importance = permutation_importance(mlp_model, X_test_scaled, y_test, n_repeats=10, random_state=42)

# Convertir los resultados a un DataFrame
importances_df = pd.DataFrame({'Variable': features, 'Importancia': perm_importance.importances_mean})
importances_df = importances_df.sort_values(by='Importancia', ascending=True)

# Graficar la importancia de las variables
plt.figure(figsize=(10, 6))
plt.barh(importances_df['Variable'], importances_df['Importancia'], color='skyblue')
plt.xlabel("Importancia")
plt.ylabel("Variables")
plt.title("Importancia de las Variables en el Modelo MLP (Permutation Importance)")
plt.show()


In [ ]:

print("Training:\n",metrics_train_mlp, "\nTest:\n",metrics_test_mlp,"\nValidation:\n", metrics_val_mlp)


Análisis:

Algunas variables tienen una mayor influencia en la predicción de la profundidad del embalse.
Al no haber una estructura clara como en los modelos de árboles, la influencia de cada variable puede ser más distribuida y dependiente de interacciones no lineales.
Las variables con mayor importancia indican que tienen una mayor contribución a la precisión del modelo.

### Guardado del mejor modelo

In [ ]:
import joblib

# Guardar el modelo en un archivo
joblib.dump(mlp_model, "mlp_regressor_model.pkl")
print("Modelo MLP guardado como 'mlp_regressor_model.pkl'")


### Carga del mejor modelo y prediccion

In [ ]:
# Cargar el modelo desde el archivo
loaded_mlp_model = joblib.load("mlp_regressor_model.pkl")

# Usar el modelo cargado para hacer predicciones
y_pred_loaded = loaded_mlp_model.predict(X_test_scaled)

# Mostrar las primeras 5 predicciones
print("Predicciones con el modelo cargado:", y_pred_loaded[:5])

### Conjunto de datos

El grupo de datos completamente independiente, que no ha intervenido en ningún momento en el entrenamiento ni en la optimización de hiperparámetros, es el conjunto de **test**.

**Entrenamiento (60%):**

Se usó para ajustar el modelo tanto en la versión inicial como en la versión optimizada con Optuna.
Participó en la estimación de los mejores valores de los hiperparámetros.

**Validación (20%):**

Se usó en Optuna para evaluar los diferentes modelos mientras se ajustaban los hiperparámetros.
Se utilizó en el cálculo del RMSE para determinar el mejor conjunto de hiperparámetros.

**Test (20%):**

No se usó en ningún momento durante el entrenamiento ni en la optimización de hiperparámetros.
Se utilizó únicamente después de obtener el mejor modelo para evaluar su capacidad de generalización en datos completamente nuevos.

## MEJORA DE LA RED NEURONAL
### Optimización con Optuna
¿Qué vamos a optimizar?
1. Número de capas ocultas (entre 1 y 4 capas).
2. Número de neuronas por capa (entre 32 y 512).
3. Tasa de regularización alpha (para evitar sobreajuste).
4. Tipo de optimizador (adam vs sgd).
5. Número de iteraciones (max_iter).

In [ ]:
# Importar Optuna
import optuna
import optuna.visualization as vis
# Definir la función de optimización con Optuna
def objective(trial):
    # Hiperparámetros a optimizar
    hidden_layer_1 = trial.suggest_int("hidden_layer_1", 32, 512)
    hidden_layer_2 = trial.suggest_int("hidden_layer_2", 32, 512)
    hidden_layer_3 = trial.suggest_int("hidden_layer_3", 32, 512)
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1,log=True)  # Regularización L2
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])
    max_iter = trial.suggest_int("max_iter", 500, 2000)

    # Definir el modelo con los hiperparámetros seleccionados
    mlp_model = MLPRegressor(hidden_layer_sizes=(hidden_layer_1, hidden_layer_2, hidden_layer_3),
                             activation='relu',
                             solver=solver,
                             alpha=alpha,
                             max_iter=max_iter,
                             random_state=42)

    # Entrenar la red neuronal
    mlp_model.fit(X_train_scaled, y_train)

    # Evaluar en el conjunto de validación
    y_val_pred = mlp_model.predict(X_val_scaled)
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))  # Minimizar RMSE

    return rmse

# Ejecutar la optimización con Optuna
study = optuna.create_study(direction="minimize")  # Minimizar el RMSE
study.optimize(objective, n_trials=25)  # Ejecutar 30 pruebas de hiperparámetros

# Obtener los mejores hiperparámetros encontrados
best_params_mlp = study.best_params

# 📊 Gráficos de análisis de Optuna
# 1️⃣ Optimization History Plot
fig1 = vis.plot_optimization_history(study)
fig1.show()

# 2️⃣ Importancia de los hiperparámetros
fig2 = vis.plot_param_importances(study)
fig2.show()

# 3️⃣ Slice Plot (impacto de cada hiperparámetro en RMSE)
fig3 = vis.plot_slice(study)
fig3.show()

# 4️⃣ Parallel Coordinate Plot (relación entre hiperparámetros y RMSE)
fig4 = vis.plot_parallel_coordinate(study)
fig4.show()

In [ ]:
# Entrenar el mejor modelo con los hiperparámetros óptimos
#best_mlp_model = MLPRegressor(**best_params_mlp, activation='relu', random_state=42)
hidden_layers = (best_params_mlp["hidden_layer_1"],
                 best_params_mlp["hidden_layer_2"],
                 best_params_mlp["hidden_layer_3"])

# Crear el modelo MLP con los mejores hiperparámetros
best_mlp_model = MLPRegressor(hidden_layer_sizes=hidden_layers,
                              activation='relu',
                              solver=best_params_mlp["solver"],
                              alpha=best_params_mlp["alpha"],
                              max_iter=best_params_mlp["max_iter"],
                              random_state=42)

best_mlp_model.fit(X_train_scaled, y_train)

# Realizar predicciones con el mejor modelo optimizado
y_train_pred_best_mlp = best_mlp_model.predict(X_train_scaled)
y_test_pred_best_mlp = best_mlp_model.predict(X_test_scaled)
y_val_pred_best_mlp = best_mlp_model.predict(X_val_scaled)

# Calcular métricas del modelo optimizado
metrics_train_best_mlp = {
    "MAE": mean_absolute_error(y_train, y_train_pred_best_mlp),
    "MSE": mean_squared_error(y_train, y_train_pred_best_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_best_mlp)),
    "R²": r2_score(y_train, y_train_pred_best_mlp)
}

metrics_test_best_mlp = {
    "MAE": mean_absolute_error(y_test, y_test_pred_best_mlp),
    "MSE": mean_squared_error(y_test, y_test_pred_best_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_best_mlp)),
    "R²": r2_score(y_test, y_test_pred_best_mlp)
}

metrics_val_best_mlp = {
    "MAE": mean_absolute_error(y_val, y_val_pred_best_mlp),
    "MSE": mean_squared_error(y_val, y_val_pred_best_mlp),
    "RMSE": np.sqrt(mean_squared_error(y_val, y_val_pred_best_mlp)),
    "R²": r2_score(y_val, y_val_pred_best_mlp)
}

# Mostrar los mejores hiperparámetros y las métricas optimizadas
print("Best Hyperpars:\n",best_params_mlp,"\nBest MLP Train:\n", metrics_train_best_mlp,"\nTest MLP Test:\n", metrics_test_best_mlp, "\nBest MLP Val:\n",metrics_val_best_mlp)


En las redes neuronales MLP (Multilayer Perceptron) no hay una métrica directa de importancia de las variables.

Para estimar la importancia de las variables en un MLP, usamos **Permutation Feature Importance**, que es una técnica basada en el impacto de cada variable en la predicción.
Proceso de cálculo:

Se calcula el RMSE original del modelo.
Se mezcla aleatoriamente una de las variables de entrada.
Se vuelve a calcular el RMSE del modelo con la variable alterada.
La importancia de la variable se define como la diferencia entre el RMSE original y el RMSE después de la permutación.
Cuanto más aumente el error al desordenar una variable, más importante es esa variable.

**¿Las importancias son relativas?**
 No son relativas ni normalizadas.
 Son valores absolutos de impacto en el error del modelo.

Si una variable tiene una importancia de 1.75, significa que al eliminar su relación con el objetivo (mezclar sus valores aleatoriamente), el error del modelo aumentó en 1.75 unidades de RMSE.

**¿Por qué "red" está por encima de 1.75?**

La variable "red" tiene un impacto fuerte en la predicción.
Si la eliminamos o la permutamos, el error del modelo aumenta significativamente.
Valores altos indican variables con mayor impacto en la predicción.
Optimization History Plot (plot_optimization_history)
📉 Muestra cómo el RMSE ha ido mejorando en cada prueba de Optuna.

Importancia de los hiperparámetros (plot_param_importances)
📊 Indica qué hiperparámetros tienen mayor impacto en el RMSE.

Slice Plot (plot_slice)
📏 Permite ver cómo varía el RMSE en función de cada hiperparámetro.

Parallel Coordinate Plot (plot_parallel_coordinate)
🔗 Muestra cómo diferentes combinaciones de hiperparámetros afectan el RMSE.


In [ ]:
# Crear un DataFrame con las coordenadas "x", "y", "z" y la profundidad predicha para cada conjunto

# Agregar las predicciones al conjunto de datos original
df_train = X_train.copy()
df_train["x"] = df_total.loc[X_train.index, "x"]
df_train["y"] = df_total.loc[X_train.index, "y"]
df_train["z"] = df_total.loc[X_train.index, "z"]
df_train["Profundidad_Predicha"] = y_train_pred_best_mlp

df_test = X_test.copy()
df_test["x"] = df_total.loc[X_test.index, "x"]
df_test["y"] = df_total.loc[X_test.index, "y"]
df_test["z"] = df_total.loc[X_test.index, "z"]
df_test["Profundidad_Predicha"] = y_test_pred_best_mlp

df_val = X_val.copy()
df_val["x"] = df_total.loc[X_val.index, "x"]
df_val["y"] = df_total.loc[X_val.index, "y"]
df_val["z"] = df_total.loc[X_val.index, "z"]
df_val["Profundidad_Predicha"] = y_val_pred_best_mlp

# Seleccionar solo las columnas deseadas
df_train = df_train[["x", "y", "z", "Profundidad_Predicha"]]
df_test = df_test[["x", "y", "z", "Profundidad_Predicha"]]
df_val = df_val[["x", "y", "z", "Profundidad_Predicha"]]

# Guardar en un archivo Excel con tres hojas
output_file = "Resultados_Predicciones.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df_train.to_excel(writer, sheet_name="Entrenamiento", index=False)
    df_test.to_excel(writer, sheet_name="Test", index=False)
    df_val.to_excel(writer, sheet_name="Validación", index=False)

# Confirmar guardado
print(f"Archivo guardado: {output_file}")


In [ ]:
# Al final del Notebook
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Tiempo total de ejecución: {elapsed_time:.2f} segundos")
from google.colab import output

# Al final del Notebook
output.eval_js('alert("Ejecución terminada 🚀")')

In [ ]:
import smtplib

def send_email():
    sender_email = "sgcortes@gmail.com"  # Cambia esto por tu email
    receiver_email = "sgcortes@gmail.com"  # También aquí
    password = "tu_contraseña"  # ⚠️ Usa una contraseña de aplicación (NO la real)

    message = "Subject: Google Colab - Ejecución terminada\n\nEl código ha finalizado 🚀"

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

send_email()